In [4]:
import os
import json
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
from google.colab import drive


In [5]:
# 구글 드라이브 마운트
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
# 폴더 경로 설정
folder_path = "/content/drive/MyDrive/특허데이터/20190115, 16, 17, 18, 21_xml_files"
output_folder = "/content/drive/MyDrive/특허데이터/data_v1"
output_file = "/content/drive/MyDrive/특허데이터/data_v1/dataset.json"

os.makedirs(output_folder, exist_ok=True)

In [21]:
# XML 파일을 JSON으로 변환하는 함수
def convert_xml_to_json(xml_file):

    # 파일을 열어서 내용 읽기 (인코딩 지정)
    with open(xml_file, "r", encoding="EUC-KR") as file:  # "utf-8" 대신 실제 인코딩으로 변경
        xml_content = file.read()
    # XML 문자열 파싱
    root = ET.fromstring(xml_content)

    # 발명 Title 추출
    title_tag = root.find(".//KR_RequestInformation/KR_InventionTitle")

    # Field, Background Art 추출
    description = root.find(".//description")
    field_paragraphs = []
    background_paragraphs = []
    if description is not None:
        field_tag = description.find("technical-field")
        if field_tag is not None:
            field_paragraphs = [p.text.strip() for p in field_tag.findall("p") if p.text]
        background_art_tag = description.find("background-art")
        if background_art_tag is not None:
            background_paragraphs = [p.text.strip() for p in background_art_tag.findall("p") if p.text]

    # Summary 추출
    summary_tag = root.find(".//abstract/summary")
    summary_text = ""
    if summary_tag is not None:
        soup = BeautifulSoup(ET.tostring(summary_tag, encoding="unicode"), "html.parser")
        summary_text = soup.get_text(separator=" ").strip()

    # KR_ApplicationNumber 추출
    application_num_tag = root.find(".//KR_ApplicationNumber")
    application_num = application_num_tag.text.replace("-", "") if application_num_tag is not None else None

    # IPC 정보 추출
    ipc_info = root.find(".//KR_Bibliographic/KR_IPCInformation")
    main_class_data = {}
    sub_class_data = []

    if ipc_info is not None:
        main_class = ipc_info.find("KR_MainClassification")
        if main_class is not None:
            full_code = main_class.text.split("\t")[0]
            main_class_data = {
                "section": full_code[0],
                "class": full_code[:3],
                "subclass": full_code.split(" ")[0]
            }
        sub_classes = ipc_info.findall("KR_SubClassification")
        for sub_class in sub_classes:
            full_code = sub_class.text.split("\t")[0]
            sub_class_data.append({
                "section": full_code[0],
                "class": full_code[:3],
                "subclass": full_code.split(" ")[0]
            })

    # JSON 데이터 생성
    json_data = {
        "application_num": application_num,
        "title": title_tag.text if title_tag is not None else None,
        "field": " ".join(field_paragraphs),
        "background_art": " ".join(background_paragraphs),
        "summary": summary_text,
        "main_classification": main_class_data,
        "sub_classification": sub_class_data
    }

    return json_data

In [22]:
# 전체 데이터를 담을 리스트
dataset = []

# 폴더 내 모든 XML 파일 변환
for file_name in os.listdir(folder_path):
    if file_name.endswith(".xml"):
        xml_path = os.path.join(folder_path, file_name)
        json_data = convert_xml_to_json(xml_path)
        dataset.append(json_data)
        print(f"추가 완료: {file_name}")

# JSON 파일 저장
with open(output_file, "w", encoding="utf-8") as json_file:
    json.dump(dataset, json_file, ensure_ascii=False, indent=4)

print(f"모든 XML 파일이 하나의 JSON 데이터셋으로 변환되었습니다: {output_file}")# 폴더 내 모든 XML 파일 변환

추가 완료: 1020170080527.xml
추가 완료: 1020170086887.xml
추가 완료: 1020170086905.xml
추가 완료: 1020170086900.xml
추가 완료: 1020170086893.xml
추가 완료: 1020170086899.xml
추가 완료: 1020170086904.xml
추가 완료: 1020170086906.xml
추가 완료: 1020170086896.xml
추가 완료: 1020170086920.xml
추가 완료: 1020170086908.xml
추가 완료: 1020170086915.xml
추가 완료: 1020170086919.xml
추가 완료: 1020170086912.xml
추가 완료: 1020170086914.xml
추가 완료: 1020170086909.xml
추가 완료: 1020170086923.xml
추가 완료: 1020170086924.xml
추가 완료: 1020170086926.xml
추가 완료: 1020170086946.xml
추가 완료: 1020170086942.xml
추가 완료: 1020170086948.xml
추가 완료: 1020170086933.xml
추가 완료: 1020170086938.xml
추가 완료: 1020170086931.xml
추가 완료: 1020170086927.xml
추가 완료: 1020170086932.xml
추가 완료: 1020170086935.xml
추가 완료: 1020170086940.xml
추가 완료: 1020170086955.xml
추가 완료: 1020170086960.xml
추가 완료: 1020170086951.xml
추가 완료: 1020170086965.xml
추가 완료: 1020170086968.xml
추가 완료: 1020170086962.xml
추가 완료: 1020170086970.xml
추가 완료: 1020170086974.xml
추가 완료: 1020170086971.xml
추가 완료: 1020170086964.xml
추가 완료: 1020170086976.xml


In [24]:
#json파일 내의 리스트 요소 개수 세기
import json

# JSON 파일 경로 설정
json_file_path = "/content/drive/MyDrive/특허데이터/data_v1/dataset_v1.json"  # 파일명을 적절히 변경해줘!

# JSON 파일 로드 및 리스트 개수 확인
with open(json_file_path, "r", encoding="utf-8") as json_file:
    data = json.load(json_file)  # JSON 데이터 로드

# 리스트 개수 출력
if isinstance(data, list):  # JSON 데이터가 리스트인지 확인
    print(f"JSON 파일 내 리스트 요소 개수: {len(data)}")
else:
    print("JSON 파일 내 데이터가 리스트 형식이 아닙니다.")


JSON 파일 내 리스트 요소 개수: 2319
